# 다양한 출력 파서 종류
- PydanticOutputParser
- JsonOutputParser
- CommaSeparatedListOutputParser
- PandasDataFrameOutputParser
- DatetimeOutputParser

## 1. Pydantic 방식으로 응답 결과 받기

### 1. 기본 모듈 및 API KEY 가져오기

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [2]:
from dotenv import load_dotenv
import os 

# API KEY 가져오기
load_dotenv()

True

### 2. Pydantic 모델 설정 // prompt 설정 | 언어모델 설정 | parser 설정

In [3]:
# pydantic 모델 설정
class GameRequest(BaseModel):
    type: str = Field(description="요청 종류")
    description: str = Field(description="요청의 상세 설명")
    expected_result: str = Field(description="요청의 기대 결과를 친절하게 설명해주세요")

In [4]:
# 언어 모델 설정
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4-turbo",  # 모델명
)

In [5]:
# 게임 유저가 서버에 보내는 요청 메시지 템플릿 생성
template = """
당신은 열성적인 게임 유저입니다. 게임 서버에 다음과 같은 요청을 보냅니다. 답변은 한글로 해주세요

요청 내용:
{request_content}

FORMAT:
{format}
"""
prompt = PromptTemplate.from_template(template=template)


In [6]:
# parser 설정
parser = PydanticOutputParser(pydantic_object=GameRequest)

In [7]:
# 프롬프트 설정
prompt = prompt.partial(format=parser.get_format_instructions())
prompt

PromptTemplate(input_variables=['request_content'], partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"type": {"title": "Type", "description": "\\uc694\\uccad \\uc885\\ub958", "type": "string"}, "description": {"title": "Description", "description": "\\uc694\\uccad\\uc758 \\uc0c1\\uc138 \\uc124\\uba85", "type": "string"}, "expected_result": {"title": "Expected Result", "description": "\\uc694\\uccad\\uc758 \\uae30\\ub300 \\uacb0\\uacfc\\ub97c \\uce5c\\uc808\\ud558\\uac8c \\uc124\\uba85\\ud574\\uc8fc\\uc138\\uc694", "type": "string"}}, "requ

In [8]:
chain = prompt | llm | parser

In [9]:
# 완성된 Chain 을 이용하여 요청 내용을 '아이템 복구 요청'으로 설정하여 실행합니다.
response = chain.invoke(
    {
        "request_content": "제 계정에서 최근 사라진 '영웅의 검' 아이템을 복구해주세요. 이 아이템은 어제 저녁에 사용 중 갑자기 사라졌습니다."
    }
)

# 결과를 딕셔너리 형태로 출력.
for item in dict(response).items():
    print(item)

('type', '복구 요청')
('description', "제 계정에서 최근 사라진 '영웅의 검' 아이템을 복구해주세요. 이 아이템은 어제 저녁에 사용 중 갑자기 사라졌습니다.")
('expected_result', "계정에서 사라진 '영웅의 검' 아이템이 성공적으로 복구되어 다시 사용할 수 있게 됩니다.")


### 그 외 파서들은 찾아보고 용도에 맞게 해봅시다. (왠만하면 Pydantic 이 좋음)

### 2. 메모리 사용법 - ConversationBufferMemory 
- 대화 내용을 그대로 저장

In [10]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context(
    inputs={
        "human": "안녕하세요 여러분"
    },
    outputs={
        "ai": "안녕하세요! 어떻게 도와드릴까요? 오늘 하루는 어떠셨나요?"
    },
)
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='안녕하세요 여러분'), AIMessage(content='안녕하세요! 어떻게 도와드릴까요? 오늘 하루는 어떠셨나요?')]))

### 3-1. ConversationChain+ConversationBufferMemory 사용법

In [12]:
from langchain import ConversationChain

In [13]:
# llm 정의
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4-turbo",  # 모델명
)

# 메모리 설정
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory
)

# 첫 대화
conversation.predict(input="안녕하세요 여러분")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕하세요 여러분
AI:

> Finished chain.


'안녕하세요! 어떻게 도와드릴까요? 오늘 하루는 어떠셨나요?'

In [14]:
# 두 번째 대화
conversation.predict(input="제가 뭐라고 했죠?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요 여러분
AI: 안녕하세요! 어떻게 도와드릴까요? 오늘 하루는 어떠셨나요?
Human: 제가 뭐라고 했죠?
AI:

> Finished chain.


'\'안녕하세요 여러분\'이라고 말씀하셨습니다. 이는 한국어로 "Hello everyone"이라는 뜻입니다. 무엇에 대해 이야기하고 싶으신가요?'

In [15]:
# 전체 대화 내용 확인
print(memory.buffer)

Human: 안녕하세요 여러분
AI: 안녕하세요! 어떻게 도와드릴까요? 오늘 하루는 어떠셨나요?
Human: 제가 뭐라고 했죠?
AI: '안녕하세요 여러분'이라고 말씀하셨습니다. 이는 한국어로 "Hello everyone"이라는 뜻입니다. 무엇에 대해 이야기하고 싶으신가요?


In [16]:
# 리스트의 형태로 출력해보기
for chat in memory.buffer_as_messages:
    print(chat.content)

안녕하세요 여러분
안녕하세요! 어떻게 도와드릴까요? 오늘 하루는 어떠셨나요?
제가 뭐라고 했죠?
'안녕하세요 여러분'이라고 말씀하셨습니다. 이는 한국어로 "Hello everyone"이라는 뜻입니다. 무엇에 대해 이야기하고 싶으신가요?


### 3-2. ConversationChain+ConversationBufferMemory 사용법

In [17]:
from langchain.memory import ConversationBufferWindowMemory

# llm 정의
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4-turbo",  # 모델명
)

# 메모리 설정
memory = ConversationBufferWindowMemory(k=2)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory
)

In [18]:
# 첫 대화
conversation.predict(input="안녕하세요 여러분! 첫번째 대화에요")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕하세요 여러분! 첫번째 대화에요
AI:

> Finished chain.


'안녕하세요! 첫 번째 대화를 함께 할 수 있어서 정말 기쁩니다. 저는 다양한 주제에 대해 이야기하는 것을 좋아해요. 어떤 주제에 대해 이야기하고 싶으신가요?'

In [19]:
# 두번째 대화
conversation.predict(input="안녕하세요 여러분! 두번째 대화에요")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요 여러분! 첫번째 대화에요
AI: 안녕하세요! 첫 번째 대화를 함께 할 수 있어서 정말 기쁩니다. 저는 다양한 주제에 대해 이야기하는 것을 좋아해요. 어떤 주제에 대해 이야기하고 싶으신가요?
Human: 안녕하세요 여러분! 두번째 대화에요
AI:

> Finished chain.


'안녕하세요! 두 번째 대화도 함께하게 되어 기쁩니다. 오늘은 어떤 주제로 이야기를 나누고 싶으신가요? 예를 들어, 최근에 관심 있는 취미나, 특별히 알고 싶은 지식이 있다면 제가 도와드릴 수 있어요!'

In [20]:
# 세번째 대화
conversation.predict(input="안녕하세요 여러분! 세번째 대화에요")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요 여러분! 첫번째 대화에요
AI: 안녕하세요! 첫 번째 대화를 함께 할 수 있어서 정말 기쁩니다. 저는 다양한 주제에 대해 이야기하는 것을 좋아해요. 어떤 주제에 대해 이야기하고 싶으신가요?
Human: 안녕하세요 여러분! 두번째 대화에요
AI: 안녕하세요! 두 번째 대화도 함께하게 되어 기쁩니다. 오늘은 어떤 주제로 이야기를 나누고 싶으신가요? 예를 들어, 최근에 관심 있는 취미나, 특별히 알고 싶은 지식이 있다면 제가 도와드릴 수 있어요!
Human: 안녕하세요 여러분! 세번째 대화에요
AI:

> Finished chain.


'안녕하세요! 세 번째 대화를 시작하게 되어 정말 반갑습니다. 오늘은 어떤 주제에 대해 깊이 있게 이야기해 볼까요? 최근에 새롭게 배우기 시작한 것이 있거나, 특별히 궁금한 점이 있으면 언제든지 말씀해 주세요. 저는 다양한 정보를 제공할 준비가 되어 있어요!'

In [21]:
print(memory.buffer) # k=2 를 입력하였으므로, 처음 대화는 메모리에서 자동 삭제

Human: 안녕하세요 여러분! 두번째 대화에요
AI: 안녕하세요! 두 번째 대화도 함께하게 되어 기쁩니다. 오늘은 어떤 주제로 이야기를 나누고 싶으신가요? 예를 들어, 최근에 관심 있는 취미나, 특별히 알고 싶은 지식이 있다면 제가 도와드릴 수 있어요!
Human: 안녕하세요 여러분! 세번째 대화에요
AI: 안녕하세요! 세 번째 대화를 시작하게 되어 정말 반갑습니다. 오늘은 어떤 주제에 대해 깊이 있게 이야기해 볼까요? 최근에 새롭게 배우기 시작한 것이 있거나, 특별히 궁금한 점이 있으면 언제든지 말씀해 주세요. 저는 다양한 정보를 제공할 준비가 되어 있어요!


In [23]:
for chat in memory.buffer_as_messages:
    print(chat.content)

안녕하세요 여러분! 두번째 대화에요
안녕하세요! 두 번째 대화도 함께하게 되어 기쁩니다. 오늘은 어떤 주제로 이야기를 나누고 싶으신가요? 예를 들어, 최근에 관심 있는 취미나, 특별히 알고 싶은 지식이 있다면 제가 도와드릴 수 있어요!
안녕하세요 여러분! 세번째 대화에요
안녕하세요! 세 번째 대화를 시작하게 되어 정말 반갑습니다. 오늘은 어떤 주제에 대해 깊이 있게 이야기해 볼까요? 최근에 새롭게 배우기 시작한 것이 있거나, 특별히 궁금한 점이 있으면 언제든지 말씀해 주세요. 저는 다양한 정보를 제공할 준비가 되어 있어요!


### 4. 대화 챗봇 만들기 - 대화 내용 기록

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 데이브 더 다이브의 게임 가이드 입니다. {number}자 내외로 한글로 답변하세요"
        ),
        
        MessagesPlaceholder(variable_name="history"), # history 이름의 메세지 저장 객체
        (
            "human",
            "{input}" # 사용자 입력을 변수로 받기
        )
    ]
)
chain = prompt | model

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

session_store = {} # 메시지 기록(세션)을 저장할 딕셔너리

# 세션 ID 를 기반으로 세션기록 가져오는 함수 만들기
def get_session_history(session_ids : str) -> BaseChatMessageHistory:
    print(session_ids)

    if session_ids not in session_store(): # 세션 기록이 없을 경우
        # 새로운 객체 생성해서 세션 스토어에 저장하기
        session_store[session_ids] = ChatMessageHistory()
    return session_store[session_ids] # 저장된 세션 ID 기록 반환하기

In [8]:
with_message_history = (
    RunnableWithMessageHistory(
        runnable=chain,
        get_session_history=get_session_history,
        input_messages_key="input",
        history_messages_key='history',
    )
)

In [10]:
result = with_message_history.invoke(
    {"number": "100", "input": "무기에 대해 알려주세요"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)
print(result)


abc123
content='무기는 주로 적을 공격하거나 방어하기 위해 사용되는 도구나 장비를 의미합니다. 종류에는 권총, 소총, 샷건, 칼, 창 등이 있으며, 군사, 경찰, 사냥 등 다양한 분야에서 사용됩니다. 안전하게 다루고 보관해야 하며, 남에게 해를 가하는 용도로 사용해서는 안 됩니다. 현지 법률을 준수해야 합니다.' response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 203, 'total_tokens': 353}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None} id='run-a77a9594-8765-4396-aefd-6c028d369522-0'


In [11]:
with_message_history.invoke(
    {"number": "50", "input": "이전의 답변 좀 더 요약해주세요"},
    config={"configurable": {"session_id": "abc123"}},
)

abc123


AIMessage(content='무기는 적을 공격하거나 방어하는 도구이며, 권총, 소총, 칼 등이 있습니다. 안전하게 다루고 보관하고 남에게 해를 가해서는 안 됩니다.', response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 378, 'total_tokens': 448}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-4d92fcc4-b0e6-4033-b5ec-2c3b0f64da5b-0')

### 5. 텍스트 임베딩

In [158]:
!pip install tiktoken chromadb # 설치 후 재실행

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/cc/63/b7d76109331318423f9cfb89bd89c99e19f5d0b47a5105439a629224d297/chromadb-0.4.24-py3-none-any.whl.metadata
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/e2/03/f3c8ba0a6b6e30d7d18c40faab90807c9bb5e9a1e3b2fe2008af624a9c97/build-1.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/cc/3d/ca311b8f79744db3f4faad8fd9140af80d34c94829d3ed1726c98cf4a611/chroma_hnswlib-0.7.3-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for posthog>=2.4.0 from https://files.pythonhosted.org/packages/6c/5f/24cb22118db0e11703b6b80ef9f982eadde21eb585c3a769719e48dce893/posthog-3.5.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pulsar-client>=3.1.0 from https://files.pythonhosted.org/packages/0a/cc/d606b483dbb263cbaf7fc7c3d2ec4032628cf3324266cf

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import OpenAIEmbeddings

In [194]:
!pip install unstructured

  Obtaining dependency information for unstructured from https://files.pythonhosted.org/packages/54/73/b9c499b669ce2387dde71968c72fc6077e2df347e7bb3c5b43fec1530f5a/unstructured-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for filetype from https://files.pythonhosted.org/packages/18/79/1b8fa1bb3568781e84c9200f951c735f3f157429f44be0495da55894d620/filetype-1.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for python-magic from https://files.pythonhosted.org/packages/6c/73/9f872cb81fc5c3bb48f7227872c28975f998f3e7c2b1c16e95e6432bbb90/python_magic-0.4.27-py2.py3-none-any.whl.metadata
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/91/c7/05a3e75143d783f2731e31929ceec6e408d3152bc5ecf24e9f6a16a438e8/lxml-5.2.1-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py

In [15]:
# DirectoryLoader 사용하여 "./*.txt" 파일을 로드합니다.

loader = DirectoryLoader(".", glob="./*.txt", show_progress=True)
docs = loader.load()

print(f"문서의 수: {len(docs)}")
for num in range(len(docs)):
    print(f"\n[metadata]\n{docs[num].metadata}\n")

100%|██████████| 7/7 [00:05<00:00,  1.24it/s]

문서의 수: 7

[metadata]
{'source': '등장인물.txt'}


[metadata]
{'source': '무기.txt'}


[metadata]
{'source': '부재료 및 조미료.txt'}


[metadata]
{'source': '식당 운영.txt'}


[metadata]
{'source': '요리.txt'}


[metadata]
{'source': '장비.txt'}


[metadata]
{'source': '지역 및 생물.txt'}



In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap=100
)
docs_spliter = text_splitter.split_documents(docs)
docs_spliter[:10]

[Document(page_content='등장인물', metadata={'source': '등장인물.txt'}),
 Document(page_content='상위 문서: 데이브 더 다이버이 문서에 스포일러가 포함되어 있습니다.이 문서가 설명하는 작품이나 인물 등에 대한 줄거리, 결말, 반전 요소 등을 직·간접적으로 포함하고 있습니다.등장인물 · 지역 및 생물 · 장비 · 무기 · 요리 · 부재료 및 조미료 · 식당 운영1. 개요2. 반쵸스시2.1. 데이브2.2. 코브라2.3. 반쵸2.4. 모모2.5. 모찌3. 주변 인물3.1. 엘리3.2. 더프3.3. 닥터 베이컨3.4. 요시에3.5. 사토상3.6. 우도3.7. 오토3.8. 마키3.9. 방랑 상인 장고4. VIP4.1. 빈센트4.2. 마이클 방4.3. 새미4.4. 챌린지◆더 쉐프4.4.1. 왕팡4.4.2. 알렉스4.4.3. 파스트로5. 어인족 마을5.1. 라모5.2. 텐진5.3. 수왐5.4. 가진5.5. 구모5.6. 미마5.7. 두와5.8. 나왕, 아왕5.9. 니아모5.10. 린첸5.11. 츠치5.12. 킹롱5.13. 다쉐5.14. 가돈6. 기타 인물6.1. 해적6.2. 존 왓슨6.3. 루이스6.4. 유이6.5. 클라라6.6. 이동 상인1. 개요[편집]데이브 더 다이버의 등장인물을 정리한 문서.2. 반쵸스시[편집]2.1. 데이브[편집]데이브Dave평상복잠수복이 게임의 주인공이자 다이버치고 배가 좀 많이 나온 아저씨이다. 어느 열대의 섬에서 집게한테 중요 부위를 찝히며 휴양을 즐기고 있다가 초밥을 마음껏 먹게 해주겠다는 코브라의 제안에 비행기까지 타고 블루홀로 오게 된다. 하지만 실상은 초밥 재료를 직접 잡아오고 저녁에는 가게 경영도 도와야 되는 상황이었고, 처음에는 당황하지만 투덜대면서도 할 일은 다 한다. [1]둔해보이는 외형과는 정반대로 작중 보여주는 모습은 세계관 최강자급의 인간병기이다. 일단 하루종일 바닷속에서 잠수한 뒤 밤에는 서빙까지 하며 중간중간 농장과 양식장 업무도 보조하는데, 게임 시스템상 이

In [44]:
# 저장할 경로 지정
DB_PATH = "./chroma_db"

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs_spliter, embeddings, persist_directory=DB_PATH)
vectorstore

In [45]:
# 저장해놓은 vector DB 불러오기
dive_db = Chroma(persist_directory="./chroma_db",
             embedding_function=embeddings)

In [51]:
query = "무기에 대해 알려줘"
retriever = dive_db.as_retriever(search_kwargs={"k":5})
tmp = retriever.get_relevant_documents(query)
print(len(tmp))
print(tmp[1].page_content)
print(tmp[0].metadata)

5
상위 문서: 데이브 더 다이버등장인물 · 지역 및 생물 · 장비 · 무기 · 요리 · 부재료 및 조미료 · 식당 운영1. 개요2. 작살총2.1. 작살촉3. 근접무기3.1. 수중 나이프4. 총기4.1. 수중 라이플4.2. 트리플 악셀4.3. 레드 스나이퍼4.4. 그레네이드 런처4.5. 점착 폭탄총4.6. 네트건4.7. 허쉬 다트4.8. 아이스건1. 개요[편집]데이브 더 다이버의 무기들을 정리한 문서.근거리무기, 작살, 총기가 있으며 작살과 총기는 탭키로 교환하여 사용 가능하다. 총은 상자를 열어서 줍거나, 업그레이드 획득 / 사용 시 즉시 장전된다.속성은 화염, 전기, 수면, 독, 마비, 빙결이 있으며 각 효과는 다음과 같다. 화염: 작살에서는 도트딜. 독보다 빠르게 사라지고 강한 데미지를 준다. 총기에서는 추가 데미지로 작용. 보스인 크로노사우루스를 제외하면 화염에 면역인 어종이 없어 실질적으로 추가 데미지나 다름없기 때문에 작살의 경우 무속성을 거르고 화염 속성 작살을 선호하는 편이다. 레벨이 높아질수록 소량의 기본 데미지와 더불어 화염 데미지가 5% 증가된다. 전기: 작살/총기 데미지의 비례한 체인 라이트닝을 건다. 즉 주변의 다른 물고기로 전기를 뿌린다. 수면: 수면에 걸린 적은 행동을 하지 않으며, 수면에 걸린 해파리나 부풀린 복어의 경우에도 닿아도 데미지를 입지 않는다. 또한 가장 중요한 것으로 갈무리가 필요 없는 소형 어종이 수면에 걸렸다면 가까이 다가가서 최고 등급으로 챙길 수 있다. 중/대형 어종의 경우에는 스토리를 진행하여 인양 드론이 해금된 이후에 인양 드론 횟수를 소모하여 인양할 수 있다. 초보의 경우 이를 모르고 수면 상태인 어류에 작살을 꽂거나 칼질을 하는 경우가 많다. 독: 도트딜. 꽤 길게 이어진다. 독 작살의 경우 데미지도 시원치 않고 독에 면역인 어류에겐 독 데미지가 없는 것이나 다름없으며, 커맨드가 마우스 빙글빙글이라 어렵기 때문에 버려지는 편. 마비: 명중한 어류의 이동 속도가 느려지며 잠깐씩 경련을 일으켜 멈춘다. 주로


In [39]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(temperature=0, model="gpt-4-turbo")

# 질문 양식 설정 - 템플릿 설정
template="""당신은 데이브 더 다이브의 게임 가이드 입니다. {context} 를 기반으로 qustion에 100자 내외로 한글로 답변하세요 
Question :{question}
"""

prompt = ChatPromptTemplate.from_template(template=template)


parser = StrOutputParser()

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join([doc.page_content for doc in docs])


# 단계 RAG 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

In [41]:
query = "무기에 대해서 알려줘"
response = rag_chain.invoke(query)
print(response)

데이브 더 다이버 게임에서 사용할 수 있는 무기는 크게 작살총, 근접무기, 총기로 나뉩니다. 작살총은 물고기나 다른 해양 생물을 사냥할 때 사용하며, 근접무기로는 수중 나이프가 있습니다. 총기류에는 수중 라이플, 트리플 악셀, 레드 스나이퍼 등이 있으며, 각각 다른 특성과 효과를 가지고 있습니다. 무기는 탭 키를 통해 교환하여 사용할 수 있으며, 업그레이드를 통해 성능을 향상시킬 수 있습니다.
